# YLE model

20-02-24

Katie Willis 

Katie.willis@imperial.ac.uk

___

This file contains functions required to simulate a model containing a Y-linked editor with and without an X-shredder at a second locus. 
___


In [142]:
"""
Takes in: 
    t = no. generations of simulation
    starting_population_post_release = a Nx8 matrix of the initial population post release
    genotypes_m = list of male genotype labels
    genotypes_f = list of female genotype labels
    ic_f_vals = Nf x Nm x Nf array containing proportion of female offspring produced by each male and female parental cross
    ic_m_vals = Nf x Nm x Nm array containing proportion of male offspring produced by each male and female parental cross
    fitness_f_vals = Vector of length Nf containing relative fitness of each female genotype
    fitness_m_vals = Vector of length Nf containing relative fitness of each female genotype
    transgenic_genotype = genotype label of the released male (if performing repeat releases)
    release_regime = vector of length t containing the number of males released each generation
    fitness_effect = "b", "a","s" defining the stage at which fitness effects are applied.
Returns a dictionary containing: 
    "simnF" = Nf x 4 x t+1 Array containing the numbers of females at each life stage in each generation
    "simnM" = Nm x 4 x t+1 Array containing the numbers of males at each life stage in each generation 
"""
function Simulate_timeseries(t,             
                    starting_population_post_release, 
                    genotypes_m,     
                    genotypes_f,      
                    ic_f_vals,
                    ic_m_vals,
                    fitness_f_vals,
                    fitness_m_vals,
                    params;
                    transgenic_genotype = false,
                    release_regime = false,
                    fitness_effect = "a")

    #Generation 0
    #define (Nx4xt) storage for male and female n. individuals (zygote, pupae, adult, repro)
    nF = Array{Float64}(undef,(size(genotypes_f)[1],4,t+1))
    nM = Array{Float64}(undef,(size(genotypes_m)[1],4,t+1))

    #store starting population from input
    nF[:,:,1]=starting_population_post_release[1]
    nM[:,:,1]=starting_population_post_release[2]

    #store no. adults in starting pop
    genotype_Nf_adult=nF[:,4,1]
    #Apply reproductibe costs in males
    if fitness_effect == "s"
        nM[:,4,1] = nM[:,4,1].*fitness_m_vals
    else
        nM[:,4,1] = nM[:,4,1]
    end
    genotype_Nm_adult=nM[:,4,1]
    #store total number of starting males
    starting_Nm = sum(genotype_Nm_adult)

    #For repeat releases, store mask to define which males are released each generation
    if release_regime != false
        release_mask = [a == transgenic_genotype for a in genotypes_m[:,1]]
    end

    #For each generation: 
    for gen in 1:t

        #For repeat releases, release adult males into population gen t-1 to generate generation t
        #And apply the reproductive fitness costs
        if (release_regime != false)
            if fitness_effect =="s"
                genotype_Nm_adult=nM[:,3,gen]
                genotype_Nm_adult[release_mask] = genotype_Nm_adult[release_mask] .+ (starting_Nm*release_regime[gen])
                #reproductive fitness costs are applied to released adults
                genotype_Nm_adult = genotype_Nm_adult.*fitness_m_vals
            else
                genotype_Nm_adult=nM[:,4,gen]
                genotype_Nm_adult[release_mask] = genotype_Nm_adult[release_mask] .+ (starting_Nm*release_regime[gen])
            end
            nM[:,4,gen] = genotype_Nm_adult
        end
        
        #Production of male and female zygotes
        #Females can lay fewer eggs according to reproductive costs of the male
        if params["fm"] > 0.0 #Currently only coded for 2-locus X-shredding case!
            Nmating_males = nM[:,3,gen]
            #Assuming all "a" genotypes carry an X-shredder
            xs_mask = [occursin("a",genotype[1]) | occursin("a",genotype[2]) for genotype in genotypes_m] 
            genotype_freqm_fitness = Nmating_males./sum(sort(Nmating_males))
            xs_males = sum(genotype_freqm_fitness[xs_mask])
            f_gen = params["f"] * (1-(xs_males*params["fm"]))
        else
            f_gen = params["f"]
        end
        n_eggs = genotype_Nf_adult.*f_gen
        n_zygotes_f = Array{Float64}(undef,length(genotypes_f_string))
        for o in 1:length(genotypes_f_string)
            n_zygotes_f[o] = sum(((ic_f_vals[:,:,o].*n_eggs)').* (genotype_Nm_adult./(sum(genotype_Nm_adult))))
        end
        n_zygotes_m = Array{Float64}(undef,length(genotypes_m_string))
        for o in 1:length(genotypes_m_string)
            n_zygotes_m[o] = sum(((ic_m_vals[:,:,o].*n_eggs)').* (genotype_Nm_adult./(sum(genotype_Nm_adult))))
        end
        nF[:,1,gen+1] = n_zygotes_f
        nM[:,1,gen+1] = n_zygotes_m

        #fitness costs are applied to zygotes if applicable
        if fitness_effect == "b"
            n_zygotes_F = n_zygotes_f.*fitness_f_vals[:]
            n_zygotes_M = n_zygotes_m.*fitness_m_vals[:]
        else
            n_zygotes_F = n_zygotes_f
            n_zygotes_M = n_zygotes_m
        end

        #Apply density dependent costs
        n_zygotes = [n_zygotes_F,n_zygotes_M]
        n_pupae = apply_density_dependence(params,n_zygotes)
        nF[:,2,gen+1] = n_pupae[1]
        nM[:,2,gen+1] = n_pupae[2]

        #fitness costs are applied to pupae if applicable
        if fitness_effect == "a"
            n_adults_F = n_pupae[1].*fitness_f_vals
            n_adults_M = n_pupae[2].*fitness_m_vals
        else
            n_adults_F = n_pupae[1]
            n_adults_M = n_pupae[2]
        end
        n_adults = [n_adults_F,n_adults_M]
        nF[:,3,gen+1] = n_adults[1]
        nM[:,3,gen+1] = n_adults[2]

        #fitness costs are applied to adults if applicable
        if fitness_effect == "s"
            n_reproadults_F = n_adults[1].*fitness_f_vals
            n_reproadults_M = n_adults[2].*fitness_m_vals
        else
            n_reproadults_F = n_adults[1]
            n_reproadults_M = n_adults[2]
        end
        n_reproadults = [n_reproadults_F,n_reproadults_M]
        nF[:,4,gen+1] = n_reproadults[1]
        nM[:,4,gen+1] = n_reproadults[2]

        #Store for no. adults are updated for generation t+1
        genotype_Nf_adult= n_reproadults[1]
        genotype_Nm_adult= n_reproadults[2]
    end

    output = Dict(
        "simnF" => nF,
        "simnM" => nM
        )
    return(output)
end

function apply_density_dependence(params, 
                                n_zygotes)
    
    n_zygotes_sum = sum(sort(n_zygotes[1])) + sum(sort(n_zygotes[2]))
    juvenile_survival_prob = params["OJ"]*(params["α"]/(params["α"]+n_zygotes_sum))
    n_zygote_dd_f = n_zygotes[1] * juvenile_survival_prob
    n_zygote_dd_m = n_zygotes[2] * juvenile_survival_prob

    return(n_zygote_dd_f,n_zygote_dd_m)
end

"""
Initiates the prerelease population of adults. 
Back-calculates the number of zygotes for a given adult population and parameter values. 
Returns a Nx8 matrix, where N is the number of genotypes, of numbers of zygote, pupae, adults and repro adults for females and males.
"""
function Initiate_prerelease_population_4lifestages(params, genotypes_m,genotypes_f)

    
    Rm = (params["f"]*params["OJ"])/2
    Nf = params["α"]*(Rm-1)/params["f"]

    #Simulation has 4 life stages: Zygote, Pupae, Adult, Repro adult
    #Define each frequency initially as 0
    starting_population_f = Array{Float64}(undef,(length(genotypes_f),4))
    starting_population_m = Array{Float64}(undef,(length(genotypes_m),4))
    fill!(starting_population_f,0)
    fill!(starting_population_m,0)
    
    #Starting popualtion are all WT
    #Assume all pupae, adults and reproadults are fully fertile and there is no mortality between stages
    starting_population_f[1,2] = Nf
    starting_population_m[1,2] = Nf
    starting_population_f[1,3] = Nf
    starting_population_m[1,3] = Nf
    starting_population_f[1,4] = Nf
    starting_population_m[1,4] = Nf
    
    #Assuming the population is at equilibrium AND the population is pre-release
    #No. zygotes
    starting_population_f[:,1] =  starting_population_f[:,3] .* params["f"] /2
    starting_population_m[:,1] =  starting_population_m[:,3] .* params["f"] /2

    return(starting_population_f,starting_population_m)
end

"""
Takes in a Nx8 matrix, where N is the number of genotypes, defining the numbers of genotypes for a population pre-release and 
returns a Nx8  matrix of numbers of each genotype after release of males. 
The frequency and genotype of the release males is defined by the input.
"""
function release_transgenic_males(starting_population,transgenic_genotype,release_freq,genotypes)
    
    x=deepcopy(starting_population)
    mask = [a == transgenic_genotype for a in genotypes][:]
    if sum(mask)>1
        print("error in assigning trangenic values")
    end
    #Here we make the assumption that all initially released individuals are fully fit
    x[mask,4] .= sum(starting_population[:,4]).*release_freq
    
    return(x)
end

release_transgenic_males

In [143]:
"""
Takes in a simulation output and calculates various summary metrics from the numbers of individuals/gametes. 
Returns a dictionary containing the same output data with additional entries. 
"""
function Simulation_processing_wrapper(output,falleles,malleles,genotypes_f,genotypes_m)
    alleles_list = falleles
    
    nF = output["simnF"]
    nM = output["simnM"]
    t = length(nF[1,1,:])

    #Make dataframe header labels
    falleles_sym = [join(t) for t in falleles]
    malleles_sym = [join(t) for t in malleles]
    header_f = Symbol.([join(t) for t in genotypes_f])
    header_m = Symbol.([join(t) for t in genotypes_m])

    #Calculate relative no. females
    pupae = [sum(nF[:,2,i]) for i in 1:t]
    adult = [sum(nF[:,3,i]) for i in 1:t]
    reproadult = [sum(nF[:,4,i]) for i in 1:t]
    total_Nf = DataFrame(hcat(pupae,adult,reproadult),["pupae","adult","reproadult"])
    relpupae = pupae./pupae[1]
    reladult = adult./adult[1]
    relreproadult = reproadult./reproadult[1]
    relative_Nf = DataFrame(hcat(relpupae,reladult,relreproadult),["pupae","adult","reproadult"])

    #Calculate allele frequencies for zygotes
    genotypenumber_zygote_female = DataFrame(Transpose(nF[:,1,:]), header_f)
    genotypenumber_zygote_male = DataFrame(Transpose(nM[:,1,:]), header_m)
    genotype_freqzf_store = freq_from_no_df(genotypenumber_zygote_female)
    genotype_freqzm_store = freq_from_no_df(genotypenumber_zygote_male)
    allele_freqzf = Find_allele_frequencies_from_genotypes_df(genotypes_f,genotype_freqzf_store,alleles_list);
    allele_freqzm = Find_allele_frequencies_from_genotypes_df(genotypes_m,genotype_freqzm_store,alleles_list);
    allele_freqz = (allele_freqzf .+ allele_freqzm) ./2
    y_freqz = Find_y_frequencies_from_genotypes_df(genotypes_m,genotype_freqzm_store)

    #Calculate allele frequencies for zygotes
    genotypenumber_reproadult_female = DataFrame(Transpose(nF[:,4,:]), header_f)
    genotypenumber_reproadult_male = DataFrame(Transpose(nM[:,4,:]), header_m)
    genotype_freqrf_store = freq_from_no_df(genotypenumber_reproadult_female)
    genotype_freqrm_store = freq_from_no_df(genotypenumber_reproadult_male)
    allele_freqrf = Find_allele_frequencies_from_genotypes_df(genotypes_f,genotype_freqrf_store,alleles_list);
    allele_freqrm = Find_allele_frequencies_from_genotypes_df(genotypes_m,genotype_freqrm_store,alleles_list);
    allele_freqr = (allele_freqrf .+ allele_freqrm) ./2
    y_freqr = Find_y_frequencies_from_genotypes_df(genotypes_m,genotype_freqrm_store)

    #save in a dictionary
    data_dict = Dict(

        "simnF" => nF,
        "simnM" => nM,
        "number_females" => total_Nf,
        "relative_number_females" => relative_Nf,
        "genotypenumber_zygote_female" => genotypenumber_zygote_female,
        "genotypenumber_zygote_male" => genotypenumber_zygote_male,
        "genotypefreq_zygote_female" => genotype_freqzf_store,
        "genotypefreq_zygote_male" => genotype_freqzm_store,
        "allelefreq_zygote" => allele_freqz,
        "allelefreq_zygote_female" => allele_freqzf,
        "allelefreq_zygote_male" => allele_freqzm,
        "yfreq_zygote" => y_freqz,
        "genotypenumber_reproadult_female" => genotypenumber_reproadult_female,
        "genotypenumber_reproadult_male" => genotypenumber_reproadult_male,
        "genotypefreq_reproadult_female" => genotype_freqrf_store,
        "genotypefreq_reproadult_male" => genotype_freqrm_store,
        "allelefreq_reproadult" => allele_freqr,
        "allelefreq_reproadult_female" => allele_freqrf,
        "allelefreq_reproadult_male" => allele_freqrm,
        "yfreq_reproadult" => y_freqr,
    );
    return(data_dict)
end

"""Calculate the row-wise frequency"""
function freq_from_no_df(genotype_N_store)
    genotype_N_store./sum(sort(Matrix(genotype_N_store),dims=2),dims=2)
end

"""
Takes in the genotype frequencies of a population at a single time point and 
calculated the allele frequency.
"""
function Find_allele_frequencies_from_genotypes(genotypes,freq_genotype,alleles_list)
    """Extract the allele frequencies from the genotype frequencies"""
    allele_freqs = Vector{Float64}()
    for allele in vcat(alleles_list...)
        allele_freq = 0
        for i in 1:length(genotypes)
            if (allele in genotypes[i][:,1]) & (allele in genotypes[i][:,2])
                allele_freq = allele_freq+freq_genotype[i]
            elseif (allele in genotypes[i][:,1]) | (allele in genotypes[i][:,2])
                allele_freq = allele_freq+(freq_genotype[i]/2)
            end
        end
       push!(allele_freqs,allele_freq)
    end
    return(allele_freqs)
end
function Find_allele_frequencies_from_genotypes_df(genotypes,genotype_freq_store,alleles_list)
    header = Symbol.(vcat(alleles_list...))
    Make_dataframe([Find_allele_frequencies_from_genotypes(genotypes,row,alleles_list) for row = eachrow(genotype_freq_store)],header)
end


function Find_y_frequencies_from_genotypes(genotypes,freq_genotype)
    """Extract the allele frequencies from the genotype frequencies"""
    allele_freqs = Vector{Float64}()
    for allele in ["Y","y"]
        allele_freq = 0
        for i in 1:length(genotypes)
            if (allele == genotypes[i][3])
                allele_freq = allele_freq+freq_genotype[i]
            end
        end
       push!(allele_freqs,allele_freq)
    end
    return(allele_freqs)
end
function Find_y_frequencies_from_genotypes_df(genotypes,genotype_freq_store)
    header = Symbol.(['Y','y'])
    Make_dataframe([Find_y_frequencies_from_genotypes(genotypes,row) for row = eachrow(genotype_freq_store)],header)
end

"""Processes a matrix into a dataframe"""
function Make_dataframe(store, header)
    df = DataFrame(Transpose(hcat(store...)),header)
end


Make_dataframe

In [70]:
function Simulate_single_release_example(subs_list,t,release_freq,release_genotype; release_regime = false,fitness_effect = "a",Rm = 6)
    
    # Substitute in values from subs_list
    
    #Fitness
    fitness_f_vals = Array{Float64}(fitness_f.subs(subs_list))
    fitness_m_vals = Array{Float64}(fitness_m.subs(subs_list))

    #inheritance
    Nf = length(ic_f[1,1,:])
    Nm = length(ic_m[1,1,:])
    ic_vals = Array{Any,3}(nothing,Nf,Nm,Nf)
    for i in 1:Nf
        ic_vals[i,:,:] = ic_f[i,:,:].subs(subs_list)
    end
    ic_f_vals = Array{Float64}(ic_vals);

    #inheritance
    ic_vals = Array{Any,3}(nothing,Nf,Nm,Nm)
    for i in 1:Nf
        ic_vals[i,:,:] = ic_m[i,:,:].subs(subs_list)
    end
    ic_m_vals = Array{Float64}(ic_vals);

    # Define pop dynamics params
    params = Dict()
    Nf_eq = 1000.0
    params["OJ"] = 1/10
    params["f"] = (2*Rm)/params["OJ"]
    params["α"] = params["f"]*Nf_eq/(Rm-1)
    params["fm"] = 0

    #Set up release
    starting_population_Nf,starting_population_Nm = Initiate_prerelease_population_4lifestages(params, genotypes_m,genotypes_f)
    if release_regime == false
        starting_population_post_release_m = release_transgenic_males(starting_population_Nm,release_genotype,release_freq,genotypes_m)
    else
        starting_population_post_release_m = starting_population_Nm
    end
    starting_population_post_release = [starting_population_Nf,starting_population_post_release_m]

    output = Simulate_timeseries(t, 
                        starting_population_post_release, #[1] = female, [2] = male
                        genotypes_m,     #list of genotypes, each entry = ["A1" "A2"; "B1" "B2"]
                        genotypes_f,     #list of genotypes, each entry = ["A1" "A2"; "B1" "B2"]) 
                        ic_f_vals,
                        ic_m_vals,
                        fitness_f_vals,
                        fitness_m_vals,
                        params,
                        fitness_effect = fitness_effect,
                        release_regime = release_regime,
                        transgenic_genotype = release_genotype)

    out = Simulation_processing_wrapper(output,falleles,malleles,genotypes_f,genotypes_m);
    return(out)
end


Simulate_single_release_example (generic function with 1 method)

In [71]:
function find_release_rate_required(n_releases,release_freq,threshold,pdict;fitness_effect = "a",release_genotype=["A" "a" "y"],param_mod=false,param_label=false,t=100,increment = 10,Rm=6)

    checker = false
    lim = 1000
    
    opt_subs_pdict = deepcopy(pdict)
    
    #mod params
    if param_mod != false
        for param in 1:length(param_mod)
            opt_subs_pdict[param_label[param]]=param_mod[param]
        end
    end
    
    opt_subs_list = [(param,opt_subs_pdict[param]) for param in keys(pdict)];
    
    #Fitness
    fitness_f_vals = Array{Float64}(fitness_f.subs(opt_subs_list))
    fitness_m_vals = Array{Float64}(fitness_m.subs(opt_subs_list))

    #inheritance
    Nf = length(ic_f[1,1,:])
    Nm = length(ic_m[1,1,:])
    ic_vals = Array{Any,3}(nothing,Nf,Nm,Nf)
    for i in 1:Nf
        ic_vals[i,:,:] = ic_f[i,:,:].subs(opt_subs_list)
    end
    ic_f_vals = Array{Float64}(ic_vals);

    #inheritance
    ic_vals = Array{Any,3}(nothing,Nf,Nm,Nm)
    for i in 1:Nf
        ic_vals[i,:,:] = ic_m[i,:,:].subs(opt_subs_list)
    end
    ic_m_vals = Array{Float64}(ic_vals);

    params = Dict()
    Nf_eq = 1000.0
    params["OJ"] = 1/10
    params["f"] = (2*Rm)/params["OJ"]
    params["α"] = params["f"]*Nf_eq/(Rm-1)
    params["fm"] = 0
    
    while checker == false
        
        release_regime = repeat([0.0],t)
        for i in 1:n_releases
            release_regime[i] = release_freq
        end

        #Set up release
        starting_population_Nf,starting_population_Nm = Initiate_prerelease_population_4lifestages(params, genotypes_m,genotypes_f)
        if release_regime == false
            starting_population_post_release_m = release_transgenic_males(starting_population_Nm,release_genotype,release_freq,genotypes_m)
        else
            starting_population_post_release_m = starting_population_Nm
        end
        starting_population_post_release = [starting_population_Nf,starting_population_post_release_m]

        output = Simulate_timeseries(t,             #number of generations
                            starting_population_post_release, #[1] = female, [2] = male
                            genotypes_m,     #list of genotypes, each entry = ["A1" "A2"; "B1" "B2"]
                            genotypes_f,     #list of genotypes, each entry = ["A1" "A2"; "B1" "B2"]) 
                            ic_f_vals,
                            ic_m_vals,
                            fitness_f_vals,
                            fitness_m_vals,
                            params,
                            fitness_effect = fitness_effect,
                            release_regime = release_regime,
                            transgenic_genotype = release_genotype)
        
        output = Simulation_processing_wrapper(output,falleles,malleles,genotypes_f,genotypes_m);
        
        if ((minimum(output["relative_number_females"].adult) > threshold) & (release_freq<lim))
            release_freq = release_freq+increment
        elseif release_freq>=lim
            checker = true
            return(release_freq)
        else
            if increment != 0.0001
                release_freq = release_freq-increment
                increment = increment/10
            else
                checker = true
                return(release_freq)
            end
        end
    end
end

find_release_rate_required (generic function with 1 method)

In [ ]:
#Assuming params defined as above
function find_release_rate_required_2locus(release_genotype,
                                    threshold,
                                    ic_f_vals,
                                    ic_m_vals,
                                    fitness_f_vals,
                                    fitness_m_vals,
                                    params;
                                    n_releases = 36,param_mod=false,param_label=false,fitness_effect = "a")
    
    t=n_releases
    increment = 10
    checker = false
    lim = 1000
    release_freq = 0.0


    while checker == false

        release_regime = repeat([0.0],t)
        for i in 1:n_releases
            release_regime[i] = release_freq
        end

        starting_population_Nf,starting_population_Nm = Initiate_prerelease_population_4lifestages(params, genotypes_m,genotypes_f)
        if release_regime == false
            starting_population_post_release_m = release_transgenic_males(starting_population_Nm,release_genotype,release_freq,genotypes_m)
        else
            starting_population_post_release_m = starting_population_Nm
        end
        starting_population_post_release = [starting_population_Nf,starting_population_post_release_m];

        output = Simulate_timeseries(t,             #number of generations
                        starting_population_post_release, #[1] = female, [2] = male
                        genotypes_m,     #list of genotypes, each entry = ["A1" "A2"; "B1" "B2"]
                        genotypes_f,     #list of genotypes, each entry = ["A1" "A2"; "B1" "B2"]) 
                        ic_f_vals,
                        ic_m_vals,
                        fitness_f_vals,
                        fitness_m_vals,
                        params;
                        release_regime = release_regime,
                        transgenic_genotype = release_genotype,
                        fitness_effect =fitness_effect)

        output = Simulation_processing_wrapper(output,falleles,malleles,genotypes_f,genotypes_m);
        if ((minimum(output["relative_number_females"].reproadult) > threshold) & (release_freq<lim))
            release_freq = release_freq+increment
        elseif release_freq>=lim
            checker = true
            #return(release_freq)
            println(release_freq)
        else
            if increment != 0.0001
                release_freq = release_freq-increment
                increment = increment/10
            else
                checker = true
            end
        end
    end
    return(release_freq)
end